In [1]:
!pip install dask --upgrade

Requirement already up-to-date: dask in /usr/local/lib/python3.6/dist-packages (2.30.0)


In [ ]:
!pip install distributed --upgrade

In [2]:
import pandas as pd
import numpy as np
import librosa
import time

from dask.distributed import Client
client = Client(n_workers=8)

from dask import delayed, compute

In [3]:
songs = pd.read_csv('all_decades_songs_V3.csv', index_col=0)

In [4]:
def librosa_pipeline(track_id):
   path = f'drive/My Drive/previews/track_{track_id}.wav'
   
   d = {}
   d['track_id'] = track_id

   #load mp3
   y, sr = librosa.load(path, mono=True, duration=30)
   
   #feature extraction
   spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
   d['spectral_centroid'] = np.mean(spec_cent)
   
   spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
   d['spectral_bandwidth'] = np.mean(spec_bw)
   
   rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
   d['rolloff'] = np.mean(rolloff)
   
   zcr = librosa.feature.zero_crossing_rate(y)
   d['zero_crossing_rate'] = np.mean(zcr)

   mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
   for i,e in zip(range(1, 21),mfcc):
     d[f'mfcc{i}'] = np.mean(e)

   chroma = ['C', 'C#', 'D','D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
   chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
   for c,p in zip(chroma,chroma_stft):
     d[c] = np.mean(p)

   return d

In [6]:
librosa_features = []

start = time.time()
for i,row in songs.iloc[:10,:].iterrows():
  features = librosa_pipeline(row['track_id'])
  librosa_features.append(features)

end = time.time()

print(end-start)


24.440731048583984


In [10]:
librosa_features = []

start_1 = time.time()
for i,row in songs.iloc[:50,:].iterrows():
  features = delayed(librosa_pipeline)(row['track_id'])
  librosa_features.append(features)
end_1 = time.time()
print(end_1 - start_1)

start = time.time()
results = compute(*librosa_features)
end = time.time()
print(end-start)

0.025854825973510742
107.85174655914307


In [18]:
librosa_features = []
start_1 = time.time()
for i,row in songs.iloc[50:,:].iterrows():
  features = delayed(librosa_pipeline)(row['track_id'])
  librosa_features.append(features)
end_1 = time.time()
print(end_1 - start_1)

0.35234785079956055


In [21]:
from dask.diagnostics import ProgressBar

start = time.time()
with ProgressBar():
  rest_of_results = compute(*librosa_features, scheduler="processes")
end = time.time()

print(end-start)

[########################################] | 100% Completed | 46min 26.7s
2786.856224298477


In [ ]:
# `results` contains the first 50 songs
# need to extend `results` with `rest_of_results`

In [22]:
len(rest_of_results)

1139

In [ ]:
results_df = pd.DataFrame(results)
results_df

In [26]:
rest_of_results_df = pd.DataFrame(rest_of_results)
rest_of_results_df

,track_id,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
0,50Dq4YN1RoHWl8l16jjM2W,2210.402501,2509.074748,5076.585969,0.102386,-26.981394,96.581074,4.780874,28.551561,-1.173089,7.277271,-3.664063,12.623801,-4.787973,5.541700,1.087453,3.052843,0.393948,4.676392,-4.752227,0.441772,-3.628090,5.494388,-3.023891,2.914122,0.285436,0.350041,0.597175,0.431138,0.455168,0.503319,0.353144,0.317707,0.299692,0.532951,0.264805,0.219747
1,50AhgOXYVm9ASB6PlKdKk0,2023.992204,2158.450694,4051.750312,0.096235,-128.175352,101.373768,-14.179106,29.800243,-10.094963,0.030619,2.325613,-7.468722,-17.170022,13.210442,-17.275361,3.819054,-11.692153,-2.299017,-4.883838,0.234534,-4.081527,7.011573,-5.494777,1.477710,0.333324,0.437554,0.255286,0.303844,0.223606,0.365441,0.320684,0.264584,0.441392,0.290990,0.372034,0.240852
2,5ZmGoux6Pu8rtfjBWtLR7w,1603.348257,1783.784026,3013.094839,0.077437,-219.038040,129.033495,1.327925,37.627146,-12.773166,-5.962239,5.464781,-4.660460,-21.584056,15.275674,-7.138539,-11.632612,0.462618,7.923553,-19.607665,-7.904058,-1.259746,-5.186941,-12.775756,-1.417733,0.364731,0.312688,0.260693,0.303861,0.406027,0.326449,0.316027,0.258096,0.243282,0.258267,0.306091,0.455645
3,5HisVcYPaIB4uHdrnW6HCt,2579.981188,2083.069161,4646.296904,0.148712,-173.385083,59.894048,-39.852013,32.174885,-6.645698,12.618968,-1.862377,-0.958555,-3.709075,3.853025,-2.516258,0.639864,-8.966128,0.159154,-4.667695,3.856097,2.014504,5.805553,-0.374998,2.548612,0.314340,0.222683,0.322825,0.382586,0.273334,0.370029,0.255369,0.387395,0.330820,0.280379,0.429721,0.278681
4,42yP5D36DoJ7cKKnhnwOZp,265.447472,520.979338,424.472433,0.012878,-589.366908,136.422744,50.494546,17.213465,14.843192,17.762663,22.230988,18.281334,12.430941,12.626093,12.262624,9.324884,6.905859,5.253704,4.469284,3.650222,1.759119,-0.010745,-0.681836,-0.542263,0.517163,0.518086,0.519854,0.502970,0.451311,0.421014,0.468893,0.610991,0.635164,0.619692,0.497001,0.462757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,7qEHsqek33rTcFNT9PFqLf,1699.612361,1959.331158,3590.490391,0.075431,-116.428549,133.410819,18.106902,18.147534,2.511711,10.989014,-11.442381,13.618697,-8.265519,-5.297553,1.978626,-1.754154,-2.131206,-1.327023,-2.612876,-0.977977,-10.235575,3.405843,-3.376870,-0.005968,0.293660,0.376200,0.243366,0.284163,0.240633,0.506280,0.295377,0.199011,0.305552,0.211868,0.252599,0.184903
1135,4l0Mvzj72xxOpRrp6h8nHi,2588.992759,2682.876381,5793.049739,0.099590,-117.563806,98.827621,12.219745,-2.001674,2.078975,-12.286803,-3.477526,-16.184541,-13.599867,-0.186366,-18.355846,-1.093232,-17.701874,-4.840264,-17.813584,-9.598831,-3.969000,3.232484,1.359523,11.180165,0.147180,0.225133,0.134064,0.206344,0.305579,0.250846,0.517511,0.220675,0.259692,0.167376,0.116067,0.170764
1136,21jGcNKet2qwijlDFuPiPb,2008.899464,2214.322865,4213.165809,0.066331,-137.722984,97.387023,-6.666629,23.809878,10.841772,13.824969,-3.222085,6.340652,-3.953210,0.637255,-2.602787,1.838847,-4.805369,-1.340489,-3.326926,-0.919985,-4.001701,-3.820201,-3.624194,-0.526576,0.465383,0.338822,0.358444,0.413890,0.663266,0.645994,0.527513,0.623138,0.449223,0.409159,0.317480,0.331946
1137,7aH5zH4TxVotW0meTNqEJj,2792.255593,2948.134766,6489.337305,0.091192,-59.332892,70.714699,16.215928,18.439658,4.969987,4.483815,7.034057,11.511323,-1.536233,19.774622,3.193232,4.430480,3.842771,13.018510,1.799879,10.214539,-0.572016,6.943958,-1.692426,6.772469,0.470255,0.516918,0.388574,0.428972,0.425122,0.528993,0.536032,0.510134,0.518986,0.545528,0.720846,0.552306


In [28]:
combined_df = results_df.append(rest_of_results_df,ignore_index=True)
combined_df

,track_id,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,C,C#,D,D#,E,F,F#,G,G#,A,A#,B
0,6ymkab3FTjiFzSJwhal59m,1732.700330,1976.417959,3299.006997,0.080147,-233.233029,117.784391,-9.914331,29.593986,14.371486,1.261772,-6.141816,10.421938,-14.463040,0.486391,0.962683,-0.056909,-3.613638,9.516816,-3.361298,2.543237,-1.590607,-0.645287,0.583569,-2.694959,0.393144,0.361112,0.288184,0.351256,0.293110,0.286315,0.224675,0.333573,0.337305,0.239154,0.287794,0.256743
1,4oP8eYnsSKJPC4VNfPB7dZ,1938.764058,2043.490664,3734.490555,0.090326,-175.438754,102.982056,-17.257513,20.428538,-11.891560,-7.287997,4.090089,-1.149130,-11.019074,9.048974,-15.640110,3.410571,-3.742533,1.396330,-2.501293,-0.075906,-1.992996,5.855088,0.709312,-1.346410,0.260129,0.230320,0.309542,0.253655,0.243261,0.194317,0.196244,0.296841,0.297882,0.364029,0.255494,0.294751
2,0fVtEGoXeRhllDU9ChQAZl,1473.449694,1657.453165,3049.281561,0.073879,-176.025679,137.705039,-28.247623,43.840292,-10.444447,20.695050,-3.791915,0.083526,0.913971,9.676494,0.078259,4.512813,-4.654082,9.536025,3.826367,2.627611,0.572115,-0.275374,-1.973071,-2.351283,0.294523,0.294830,0.417634,0.289207,0.364233,0.257641,0.317754,0.417593,0.286384,0.436918,0.318095,0.362618
3,7Jf323ttHKUnPylFWiaGl3,1645.497343,1772.441690,3272.838058,0.066668,-212.501610,129.751534,-25.503920,41.971660,-6.389451,17.067045,-16.040134,6.688896,-2.852949,0.896689,-5.336751,1.596038,2.912520,-3.175327,-0.324117,0.742331,-4.737736,-2.403536,-1.144542,-2.678809,0.256598,0.284801,0.381595,0.306315,0.363529,0.469970,0.346257,0.380135,0.326079,0.391443,0.419387,0.282145
4,0lO5EKoz1Rb1pJoPoldE4D,2352.875709,2301.634767,4811.802220,0.110896,-153.961912,91.617046,-22.878960,17.333832,-9.858428,5.682141,-2.386624,11.314617,-3.242631,9.784292,-4.209345,4.785483,-3.587804,3.025462,-3.611476,-0.244491,-3.691920,-1.814111,-5.706012,0.900758,0.237268,0.317759,0.440209,0.281391,0.405444,0.227974,0.308711,0.397539,0.262201,0.352514,0.208670,0.333390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,7qEHsqek33rTcFNT9PFqLf,1699.612361,1959.331158,3590.490391,0.075431,-116.428549,133.410819,18.106902,18.147534,2.511711,10.989014,-11.442381,13.618697,-8.265519,-5.297553,1.978626,-1.754154,-2.131206,-1.327023,-2.612876,-0.977977,-10.235575,3.405843,-3.376870,-0.005968,0.293660,0.376200,0.243366,0.284163,0.240633,0.506280,0.295377,0.199011,0.305552,0.211868,0.252599,0.184903
1185,4l0Mvzj72xxOpRrp6h8nHi,2588.992759,2682.876381,5793.049739,0.099590,-117.563806,98.827621,12.219745,-2.001674,2.078975,-12.286803,-3.477526,-16.184541,-13.599867,-0.186366,-18.355846,-1.093232,-17.701874,-4.840264,-17.813584,-9.598831,-3.969000,3.232484,1.359523,11.180165,0.147180,0.225133,0.134064,0.206344,0.305579,0.250846,0.517511,0.220675,0.259692,0.167376,0.116067,0.170764
1186,21jGcNKet2qwijlDFuPiPb,2008.899464,2214.322865,4213.165809,0.066331,-137.722984,97.387023,-6.666629,23.809878,10.841772,13.824969,-3.222085,6.340652,-3.953210,0.637255,-2.602787,1.838847,-4.805369,-1.340489,-3.326926,-0.919985,-4.001701,-3.820201,-3.624194,-0.526576,0.465383,0.338822,0.358444,0.413890,0.663266,0.645994,0.527513,0.623138,0.449223,0.409159,0.317480,0.331946
1187,7aH5zH4TxVotW0meTNqEJj,2792.255593,2948.134766,6489.337305,0.091192,-59.332892,70.714699,16.215928,18.439658,4.969987,4.483815,7.034057,11.511323,-1.536233,19.774622,3.193232,4.430480,3.842771,13.018510,1.799879,10.214539,-0.572016,6.943958,-1.692426,6.772469,0.470255,0.516918,0.388574,0.428972,0.425122,0.528993,0.536032,0.510134,0.518986,0.545528,0.720846,0.552306


In [29]:
combined_df.to_csv('librosa_features.csv')